In [2]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
class LayerNormalization(keras.layers.Layer):
    def __init__(self, eps=0.001, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps

    def build(self, batch_input_shape):
        self.alpha = self.add_weight(
            name="alpha", shape=batch_input_shape[-1:],
            initializer="ones")
        self.beta = self.add_weight(
            name="beta", shape=batch_input_shape[-1:],
            initializer="zeros")
        super().build(batch_input_shape) # must be at the end

    def call(self, X):
        mean, variance = tf.nn.moments(X, axes=-1, keepdims=True)
        return self.alpha * (X - mean) / (tf.sqrt(variance + self.eps)) + self.beta

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "eps": self.eps}

In [8]:
from keras.datasets import fashion_mnist

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full.astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test.astype(np.float32) / 255.

In [13]:
# Import libraries
from tqdm import tqdm
from ipywidgets import IntSlider, Button

# Define a function that performs some calculations
def calculate_something(n_iterations):
    for _ in tqdm(range(n_iterations)):
        # Perform your actual calculations here
        pass

# Create an ipywidgets slider to control the number of iterations
n_iter_slider = IntSlider(
    min=10, max=1000, value=100, description="Number of iterations"
)

# Create an ipywidgets button to trigger the calculation
calculate_button = Button(description="Calculate")

# Display the widgets
display(n_iter_slider, calculate_button)

# Bind the button click event to the calculation function
calculate_button.on_click(lambda b: calculate_something(n_iter_slider.value))

# This will show a progress bar with the number of iterations specified by the slider


IntSlider(value=100, description='Number of iterations', max=1000, min=10)

Button(description='Calculate', style=ButtonStyle())

In [14]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax"),
])

n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.sparse_categorical_crossentropy
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.SparseCategoricalAccuracy()]

In [19]:
from collections import OrderedDict
from tqdm import trange

def random_batch(X,y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx],y[idx]

with trange(1, n_epochs + 1, desc="All epochs") as epochs:
    for epoch in epochs:
        with trange(1, n_steps + 1, desc="Epoch {}/{}".format(epoch, n_epochs)) as steps:
            for step in steps:
                X_batch, y_batch = random_batch(X_train, y_train)
                with tf.GradientTape() as tape:
                    y_pred = model(X_batch)
                    main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
                    loss = tf.add_n([main_loss] + model.losses)
                gradients = tape.gradient(loss, model.trainable_variables)
                optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                for variable in model.variables:
                    if variable.constraint is not None:
                        variable.assign(variable.constraint(variable))                    
                status = OrderedDict()
                mean_loss(loss)
                status["loss"] = mean_loss.result().numpy()
                for metric in metrics:
                    metric(y_batch, y_pred)
                    status[metric.name] = metric.result().numpy()
                steps.set_postfix(status)
            y_pred = model(X_valid)
            status["val_loss"] = np.mean(loss_fn(y_valid, y_pred))
            status["val_accuracy"] = np.mean(keras.metrics.sparse_categorical_accuracy(
                tf.constant(y_valid, dtype=np.float32), y_pred))
            steps.set_postfix(status)
        for metric in [mean_loss] + metrics:
            metric.reset_states()

Epoch 1/5:   0%|                                       | 0/1718 [00:00<?, ?it/s]
Epoch 1/5:   0%| | 0/1718 [00:00<?, ?it/s, loss=2.46, sparse_categorical_accurac
Epoch 1/5:   0%| | 1/1718 [00:00<14:32,  1.97it/s, loss=2.46, sparse_categorical
Epoch 1/5:   0%| | 1/1718 [00:01<14:32,  1.97it/s, loss=3.46, sparse_categorical
Epoch 1/5:   0%| | 2/1718 [00:01<14:51,  1.92it/s, loss=3.46, sparse_categorical
Epoch 1/5:   0%| | 2/1718 [00:01<14:51,  1.92it/s, loss=3.09, sparse_categorical
Epoch 1/5:   0%| | 3/1718 [00:01<14:53,  1.92it/s, loss=3.09, sparse_categorical
Epoch 1/5:   0%| | 3/1718 [00:02<14:53,  1.92it/s, loss=2.94, sparse_categorical
Epoch 1/5:   0%| | 4/1718 [00:02<15:13,  1.88it/s, loss=2.94, sparse_categorical
Epoch 1/5:   0%| | 4/1718 [00:02<15:13,  1.88it/s, loss=2.66, sparse_categorical
Epoch 1/5:   0%| | 5/1718 [00:02<14:46,  1.93it/s, loss=2.66, sparse_categorical
Epoch 1/5:   0%| | 5/1718 [00:03<14:46,  1.93it/s, loss=2.48, sparse_categorical
Epoch 1/5:   0%| | 6/1718 [0

KeyboardInterrupt: 